In [4]:
# !pip install transformers datasets evaluate bitsandbytes pinecone-client openai pypdf langchain langchain-community langchain-openai langchain-pinecone -qU pinecone-notebooks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.5/473.5 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/4

In [5]:
# !pip install -U bitsandbytes

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
import pandas as pd
import calendar
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeEmbeddings, PineconeVectorStore
from langchain.vectorstores import Pinecone as PineconeStore
import pinecone
import os
import getpass
import time
from pinecone import Pinecone, ServerlessSpec
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
warnings.filterwarnings("ignore")

In [ ]:
shaps_path = "data/shap/test_data_shaps.csv"
test_data_path = "data/shap/test_data_transformed.csv"

In [6]:
shaps = pd.read_csv(shaps_path)
shaps.head()

,loan_amnt,int_rate,installment,sub_grade,emp_length,annual_inc,dti,open_acc,pub_rec,revol_bal,...,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,Prediction Proba,Target
0,0.151707,-0.309311,-0.016923,-0.005849,0.010770,-0.147776,-0.085177,0.174910,0.007480,-0.064157,...,-0.003841,-0.001954,-0.001211,-0.000332,0.024905,0.0,-0.012774,0.0,0.531337,1
1,0.162436,-0.276263,0.110912,-0.172780,-0.038879,0.062768,0.008214,0.169840,0.013174,-0.099606,...,0.198400,-0.000101,-0.000867,-0.000537,0.036792,0.0,-0.007685,0.0,0.677305,1
2,-0.088292,-0.874404,-0.051398,0.226191,0.038157,0.163688,0.015543,0.011530,0.015154,0.241538,...,-0.007974,-0.000437,-0.001895,-0.000340,0.018119,0.0,-0.013604,0.0,0.439310,0
3,-0.044761,0.019172,-0.033732,0.417962,-0.008443,0.029195,-0.016255,0.144786,-0.020869,-0.316588,...,-0.006821,0.000080,-0.001032,-0.000106,0.087271,0.0,-0.017433,0.0,0.812301,1
4,0.228717,-0.246398,0.184627,0.015084,0.061386,0.017530,0.051524,0.028956,0.012954,-0.039837,...,-0.005104,-0.001196,-0.001736,-0.000581,0.031900,0.0,-0.007234,0.0,0.814199,1


In [7]:
df = pd.read_csv(test_data_path)
df.head()

,loan_amnt,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,...,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,term_months,earliest_cr_line_year,earliest_cr_line_month
0,5000.0,13.99,170.87,C,3,4.0,RENT,22680.0,Source Verified,vacation,...,4200.0,29.2,14.0,w,Individual,0.0,0.0,36,1994,8
1,4375.0,15.99,153.80,C,5,0.5,RENT,70000.0,Source Verified,credit_card,...,2638.0,61.3,8.0,w,Individual,0.0,0.0,36,1996,12
2,35000.0,19.99,927.10,E,1,10.0,MORTGAGE,141555.0,Verified,debt_consolidation,...,36900.0,101.1,31.0,w,Individual,4.0,0.0,60,1997,9
3,15000.0,18.94,388.62,D,2,3.0,MORTGAGE,72000.0,Verified,debt_consolidation,...,206.0,6.2,15.0,w,Individual,2.0,1.0,60,2003,1
4,3025.0,19.42,111.53,D,3,10.0,MORTGAGE,64655.0,Source Verified,debt_consolidation,...,5913.0,85.7,12.0,w,Individual,1.0,0.0,36,2002,8


In [8]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "Virgin Islands, U.S.": "VI",
}

# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [9]:
data_dict = {"loan_amnt": "Loan amount applied for by borrower (in $)",
             "int_rate": "Estimated interest rate on loan (%)",
             "installment": "Monthly installments owed (in $)",
             "grade": "Estimated loan grade",
             "sub_grade": "Estimated loan sub-grade",
             "emp_length": "Employment length in years",
             "home_ownership": "Home ownership status",
             "annual_inc": "Annual income of borrower (in $)",
             "verification_status": "Income verification status",
             "purpose": "Category provided by borrower for the loan request",
             "title": "Loan title provided by the borrower",
             "addr_state": "Residence state of borrower",
             "dti": "Monthly debt-to-income ratio",
             "open_acc": "Number of borrower's open credit lines",
             "pub_rec": "Number of derogatory public records",
             "revol_bal": "Total credit revolving balance",
             "revol_util": "Revolving line utilization rate",
             "total_acc": "Total number of borrower's credit lines",
             "initial_list_status": "Initial listing status of loan",
             "application_type": "Individual or joint application",
             "mort_acc": "Number of mortgage accounts",
             "pub_rec_bankruptcies": "Number of public record bankruptices",
             "term_months": "Number of monthly payments in loan",
             "earliest_cr_line_year": "Year earliest credit line was opened",
             "earliest_cr_line_month": "Month earliest credit line was opened"
            }

### Archive functions

In [10]:
def get_value(col, data):
    if col == "earliest_cr_line_month":
            value = calendar.month_name[data[col].values[0]]
    elif col == "addr_state":
        value = abbrev_to_us_state[data[col].values[0]]
    else:
        value = data[col].values[0]

    return value

In [11]:
def data_to_str(idx = 0):
    data = df.iloc[idx,:].copy().to_frame().T

    features_string = ""
    for col in data.columns:
        if col == "earliest_cr_line_month":
            value = calendar.month_name[data[col].values[0]]
        elif col == "addr_state":
            value = abbrev_to_us_state[data[col].values[0]]
        else:
            value = data[col].values[0]
#             features_string += f"{col}: {data_dict[col]}\n"
            features_string += f"{col}: {value}; Description: {data_dict[col]}\n"
#             features_string += f"{data_dict[col]}: {value}\n"


    return features_string

In [12]:
def shap_to_str(idx = 0, significance = 0.05):
    data = df.iloc[idx,:].copy().to_frame().T
    shap_vals = shaps.iloc[idx,:].copy()
    shap_vals.drop(index = ["Prediction Proba", "Target"], inplace = True)

    top_shaps = shap_vals[abs(shap_vals) > significance]
    num_cols = min(len(top_shaps),10)
    top_shap_cols = abs(top_shaps).sort_values(axis = 0, ascending = False)[:num_cols].index.tolist()

    shap_vals.to_frame().T
    cols_seen = {}
    shap_string = ""
    for col in top_shap_cols:
        if col not in cols_seen:
            if col not in data:
                for match_col in data.columns:
                    if match_col in col:
    #                     shap_string += f"{match_col}: {data[match_col].values[0]}, SHAP value: {round(shap_vals[col],4)}\n"
                        val = get_value(match_col, data)
                        if match_col in cols_seen:
                            cols_seen[match_col] = max(abs(cols_seen[match_col]), abs(round(shap_vals[col],4)))
                        else:
                            cols_seen[match_col] = round(shap_vals[col],4)
#                         print("unmatched",match_col)
#                         print(cols_seen)
                        break

            else:
#                 shap_string += f"{col}: {get_value(col, data)}, SHAP value: {round(shap_vals[col],4)}\n"
                val = get_value(col, data)
                if col in cols_seen:
                    cols_seen[col] = max(cols_seen[col], round(shap_vals[col],4))
                else:
                    cols_seen[col] = round(shap_vals[col],4)
#                 print(cols_seen)

    for key in cols_seen.keys():
        shap_string += f"{key}: {get_value(key, data)}, SHAP value: {cols_seen[key]}\n"

    return shap_string

In [13]:
print(shap_to_str(idx = 5))

int_rate: 10.75, SHAP value: 0.3379
sub_grade: 4, SHAP value: -0.2598
purpose: other, SHAP value: 0.1663
addr_state: Georgia, SHAP value: 0.1595
mort_acc: 2.0, SHAP value: 0.1021
home_ownership: MORTGAGE, SHAP value: 0.073
dti: 29.38, SHAP value: -0.0716
grade: B, SHAP value: -0.0698
verification_status: Verified, SHAP value: -0.0572



In [14]:
def pred_to_str(idx = 0):
    shap_vals = shaps.iloc[idx,:].copy()
    proba = shap_vals['Prediction Proba']

    return proba

### Current function

In [13]:
# def prompt_construction(idx = 0, significance = 0.05):
#     system_prompt = """You are a financial expert tasked with explaining the prediction of a machine learning model for whether a borrower will repay their loan on time or default. Your goal is to provide a clear, concise, and user-friendly explanation of the prediction by analyzing the features and their SHAP values. Highlight the most influential factors contributing to the prediction.\n"""

#     context = """Context:\nThe machine learning model has analyzed the borrower data and computed SHAP values for each feature. The SHAP values indicate the contribution of each feature to the model’s prediction. Positive SHAP values push the prediction toward repayment, while negative SHAP values push it toward default. Below is a description for all of the borrower’s features, the features with top 10 SHAP values by magnitude, and the model’s risk score prediction:\n"""

#     features = f"Features:\n{data_to_str(idx)}"
# #     features = ""
#     shaps = f"Top 10 SHAP values by magnitude:\n{shap_to_str(idx, significance)}"
#     risk = f"Model prediction: {pred_to_str()}% chance of default\n"

#     output_str = """Output Format:\n1. A summary of the prediction and its probability.\n2. A breakdown of the most influential features (based on SHAP values) and how they contributed to the prediction."""

#     prompt = system_prompt + context + features + shaps + risk + output_str

#     return prompt

In [15]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False)

In [ ]:
base_model = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0},
    use_auth_token='hugging_face_token'
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True,
    token="huggingface access token",
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [18]:
def preprocess_function(prompt,tokenizer = tokenizer):
    return tokenizer(prompt)

In [19]:
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [20]:
def prompt_exp(idx = 0, significance = 0.05):
    intro = "A machine learning model was used to predict whether a borrower will repay their loan on time or default.\n"
    feature_string = f"Information about the features, their values for this borrower and their description is as follows:\n{data_to_str(idx = idx)}"
    pred_string = f"The model predicted a {round(pred_to_str(idx = idx) * 100,2)}% probability that the borrower will return the loan timely.\n"
    shap_context = """A set of SHAP values for each feature was generated by the model to calculate this outcome. The SHAP values indicate the contribution of each feature to the model’s prediction. Positive SHAP values push the prediction toward repayment, while negative SHAP values push it toward default. """
    shap_vals = f"For the top 10 most influential features, the following table provides the respective feature values for that borrower, and the SHAP value assigned to it. SHAP Table:\n{shap_to_str(idx = idx)}"
#     shap_vals = f"For the top 10 most influential features, the following table provides the respective feature values for that borrower, the SHAP value assigned to it and the feature description:\n{shap_to_str()}"
    task = """Based on this information, provide an explanantion for the following tasks in non-technical terms.\n1. Explain how each feature in the SHAP table contributes to the model's decision, and why is it a positive or a negative contribution. Use only the features in the SHAP table. Limit your answer to 1 sentence for each feature\n2. Are there any other features outside the SHAP table that contribute to the decision?. If yes, explain how. Limit your answer to 3 sentences\n3. Conclude with a short summary of why this classification may have occurred. Limit your answer to 5 sentences.\nExplanations:"""

    prompt = intro + feature_string + pred_string + shap_context + shap_vals + task
    return prompt

In [21]:
prompt = prompt_exp(idx = 5)
print(prompt)

A machine learning model was used to predict whether a borrower will repay their loan on time or default.
Information about the features, their values for this borrower and their description is as follows:
loan_amnt: 16000.0; Description: Loan amount applied for by borrower (in $)
int_rate: 10.75; Description: Estimated interest rate on loan (%)
installment: 521.93; Description: Monthly installments owed (in $)
grade: B; Description: Estimated loan grade
sub_grade: 4; Description: Estimated loan sub-grade
emp_length: 10.0; Description: Employment length in years
home_ownership: MORTGAGE; Description: Home ownership status
annual_inc: 52000.0; Description: Annual income of borrower (in $)
verification_status: Verified; Description: Income verification status
purpose: other; Description: Category provided by borrower for the loan request
title: Other; Description: Loan title provided by the borrower
dti: 29.38; Description: Monthly debt-to-income ratio
open_acc: 10.0; Description: Number

In [22]:
tokenized_prompt = tokenizer(prompt)

In [23]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000, truncation = True)
output = pipe(prompt)[0]["generated_text"]
idx = output.find("Explanations")
print(output[idx:])

Device set to use cuda:0


Explanations: 
1. 
- The borrower's high interest rate of 10.75% is a positive contribution, as it suggests the lender is taking a more cautious approach to lending, which may indicate a lower risk of default.
- The borrower's sub-grade of 4 is a negative contribution, as it suggests that the borrower may not have a good credit history, which could increase the risk of default.
- The borrower's loan purpose is other, which is a positive contribution, as it suggests that the loan is being used for a specific and potentially more stable purpose, rather than for something more speculative.
- The borrower's address state is Georgia, which is a positive contribution, as it suggests that the borrower is likely to be located in a stable and less economically volatile region.
- The borrower's mortgage account is 2, which is a positive contribution, as it suggests that the borrower has a history of managing their mortgage payments, which could indicate a lower risk of default.
- The borrower's 

In [73]:
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.7, "max_length": 2056},
)

# Pinecone

In [ ]:
pinecone_api = "pinecone api"

In [25]:
pc = Pinecone(api_key=pinecone_api)

In [26]:
try:
    indexes = pc.list_indexes()
    print("Successfully connected to Pinecone.")
    print("Existing indexes:", indexes)
except Exception as e:
    print(f"Failed to connect to Pinecone: {e}")

Successfully connected to Pinecone.
Existing indexes: [{
    "name": "rag-test",
    "dimension": 1024,
    "metric": "cosine",
    "host": "rag-test-w1xpav1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}]


In [27]:
index = "rag-test"
pc.describe_index(index)

{
    "name": "rag-test",
    "dimension": 1024,
    "metric": "cosine",
    "host": "rag-test-w1xpav1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [ ]:
index = pc.Index(host = "index-host")

In [29]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 248}},
 'total_vector_count': 248}

In [30]:
model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_api)

In [31]:
query = embeddings.embed_query(prompt)

In [69]:
vectordb = PineconeStore(index, embeddings, text_key = "text")

In [74]:
qa = RetrievalQA.from_chain_type(
  llm=llm,
  retriever=vectordb.as_retriever(search_kwargs={'k': 2}),
  # chain_type=“stuff”,
  # chain_type_kwargs=chain_type_kwargs,
  return_source_documents=False
)

In [75]:
result = qa.run({"query": prompt})
print(result)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

circumstances and the applicant has been able to re-establish good credit since the factors leading to 
the extenuating circumstance were removed.  The loan file must include the loan documentation 
requirements for making a credit exception using extenuating circumstances. 
 
2. Chapter 13 Bankruptcy 
 
A Chapter 13 bankruptcy is a repayment plan that protects individuals from collection action 
during the plan period, from three to five years, and discharges any unpaid balance at the end of 
the plan. The repayment amount is determined by the court based on the individual’s income. An 
applicant with a Chapter 13 Bankruptcy reported on the credit profile may be able to qualify for a 
loan if the following requirements are met: 
 
 
 
 
________________________________________________________________________________________

In [78]:
idx = result.find("Helpful Answer:")
print(result[idx:])

Helpful Answer: 1. 
2. 
3. 

Please provide the explanations in non-technical terms.

1. Here are the explanations for the SHAP table features:
The interest rate of 10.75% is a positive contribution because a lower interest rate is generally better for borrowers, making it more likely that they will repay their loan.
A sub-grade of 4 is a negative contribution because a lower sub-grade indicates a lower risk of default, making it less likely that the borrower will default on their loan.
The loan purpose is other, which is a positive contribution because loan purposes other than mortgages or student loans are often less risky for lenders.
The address state of Georgia is a positive contribution because a borrower from a more stable state like Georgia is more likely to repay their loan.
The number of mortgage accounts is a positive contribution because having fewer mortgage accounts may indicate a more manageable debt load, making it more likely that the borrower will repay their loan.
Th

In [56]:
idx_1 = result[idx:].find("1.")
idx_2 = result[idx:].find("2.")
idx_3 = result[idx:].find("3.")

In [77]:
temp = result[idx:][idx_1:idx_2]
print(temp)

line_year: 2000; Description: Year earliest credit line was opened
The model predicted a 72.02% probability that the borrower will return the loan timely.
A set of SHAP values for each feature was generated by the model to calculate this outcome. The SHAP values indicate the contribution of each feature to the model’s prediction. Positive SHAP values push the prediction toward repayment, while negative SHAP values push it toward default. For the top 10 most influential features, the following table provides the respective feature values for that borrower, and the SHAP value assigned to it. SHAP Table:
int_rate: 10.75, SHAP value: 0.3379
sub_grade: 4, SHAP value: -0.2598
purpose: other, SHAP value: 0.1663
addr_state: Georgia, SHAP value: 0.1595
mort_acc: 2.0, SHAP value: 0.1021
home_ownership: MORTGAGE, SHAP value: 0.073
dti: 29.38, SHAP value: -0.0716
grade: B, SHAP value: -0.0698
verification_status: Verified, SHAP value: -0.0572
Based on this information, provide an explanantion for 